In [57]:
import os

import pandas as pd
import MeCab

import glob
import os
import urllib.request ,urllib.error #urlを読み込むためのライブラリ
from wordcloud import WordCloud # ワードクラウドのライブラリ
import matplotlib.pyplot as plt

import plotly.express as px
import seaborn as sns
from IPython.display import display

In [58]:
# 文字起こしされたcsvファイルを保存しているフォルダからcsvファイルのpathを取得
def csv_file_path_getter(csv_folder):
    path_name = glob.glob(csv_folder)
    print('pathの数は===', len(path_name))
    return path_name

In [59]:
# csv fileの名前を取得する関数
def csv_name_getter(csv_folder):
    name_list = []
    file_name = os.listdir(csv_folder)
    for item in file_name:
        # pathのところからcsvファイルの名前だけを取得
        name_list.append(os.path.basename(item).split('.',1)[0])
    return name_list

In [60]:
# データフレームを作成して、空白行を削除したものを返す関数
def create_data_frame(csv_path):
    dataframe = pd.read_csv(csv_path)
    print('-'*20)
    print(dataframe.isnull().sum())
    print('-'*20)
    # 必要な行だけを抽出
    dataframe = dataframe[['text', 'tag', 'timestamp']]
    # text列にNanで格納されている行は削除する
    dataframe = dataframe.dropna(subset=['text'])
    # インデックス番号を振り直し
    dataframe = dataframe.reset_index(drop=True)
    # 念の為確認する
    print('-'*20)
    print(dataframe.isnull().sum())
    print('-'*20)
    print('dataframe length is ===', len(dataframe))
    return dataframe

In [61]:
# 形態素解析を実施して、動詞、名詞、形容詞を取得する関数
def word_separation_sentence_getter(dataframe):
    # mecabを準備
    # 固有名詞や新規単語に強いneologdを利用しています
    mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

    sentence_list = []
    for text in dataframe['text']:
        noun_list = [] # 名詞を格納するリスト
        verb_list = [] # 動詞を格納するリスト
        adjective_list = []  # 形容詞を格納するリスト
        mecab.parse('')
        node = mecab.parseToNode(text)
        #print('now text is ===',text)
        while node:
            # 品詞が名詞なら
            if node.feature.split(',')[0] == '名詞':
                # print('-'*20)
                # print(node.surface, '++++++',node.feature)
                noun_list.append(node.surface)
            # 品詞が動詞なら
            elif node.feature.split(',')[0] == '動詞':
                # print('#'*20)
                # print(node.surface, '++++++',node.feature)
                verb_list.append(node.surface)
            # 品詞が形容詞なら
            elif node.feature.split(',')[0] == '形容詞':
                # print('#'*20)
                # print(node.surface, '++++++',node.feature)
                adjective_list.append(node.surface)
            else:pass
            # 書き忘れると無限ループになるので注意！
            node = node.next

        item_list = [noun_list, verb_list, adjective_list]
        sentence_list.append(item_list)
    print('分かち書きと単語の格納が終了しました')
    return sentence_list

In [62]:
# 分かち書きして取得した品詞たちを','→' '空白で区切って格納する
def append_dataframe(dataframe, sentence_list):
    dataframe[['名詞', '動詞', '形容詞']] =sentence_list
    for column in dataframe.columns.values:
        # デフォルトであった列は空白で区切る必要がないので、それ以外のカラム名を特定するためのif文
        if column != 'text' and column != 'tag' and column != 'timestamp':
            dataframe[column] = dataframe[column].apply(' '.join)
    return dataframe

In [63]:
# ワードクラウドで表現したい品詞をデータフレームから抽出する関数
def choose_part_of_speech(dataframe, part):
    part_of_speech_list = []
    for word in dataframe[part]:
        part_of_speech_list.append(word)

    return part_of_speech_list

In [64]:
# リストを文字列に変換する関数
def join_list_str(list):
    return ' '.join(list)

In [65]:
# ストップワードを準備する関数
def create_stop_words():

    sloth_lib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    sloth_lib_file = urllib.request.urlopen(sloth_lib_path)
    # 読み込んだサイトからストップワードを読み込んで、utf-8でデコードしてから1単語ずつに分けて格納する
    sloth_lib_stop_words = [line.decode('utf-8').strip() for line in sloth_lib_file]
    # 格納した単語のうちu''(空白の文字列を除外する)
    sloth_lib_stop_words = [ss for ss in sloth_lib_stop_words if not ss==u'']

    stop_words_list = []
    f = open('more_stop_word.txt')
    txt_file = f.readlines()
    f.close()
    # 自分用意した追加のストップワードをmore_stop_wordに格納する
    more_stop_word = [line.strip() for line in txt_file]
    # 自分で用意したストップワードの中に空白文字列を除外して格納
    more_stop_word = [ss for ss in more_stop_word if not ss==u'']

    stop_words_list += more_stop_word
    stop_words_list += sloth_lib_stop_words
    # 重複があった場合削除するため
    stop_words_list = set(stop_words_list)

    return stop_words_list

In [66]:
# ストップワードを除外する関数
def drop_stop_words(text, stopwords):
    change_text = []
    for token in text.lower().split(" "):
        if token != "":
            if token not in stopwords:
                change_text.append(token)
            # else:
            #     print('この単語は除外します=====',token) # 除外したい単語が見たい時はonにしてください
    return change_text

In [67]:
# ワードクラウドを書き出す関数
def create_word_cloud(word_list, title):
    font_path = '/Library/Fonts/Arial Unicode.ttf'
    word_cloud = WordCloud(
                        width=480,
                        height=320,
                        background_color='white',
                        font_path=font_path,
                        max_words = 200,
                        max_font_size=100,
                        random_state=42,
    )
    word_cloud.generate(word_list)
    plt.figure(figsize=(12,12))
    plt.title(title)
    plt.imshow(word_cloud)

In [68]:
# 単語をカウントしてデータフレームで返す関数
def count_word(text_list):
    # 空の辞書を準備
    word_freq_dic = {}
    for word in text_list:
        #　もし、辞書に単語が存在していたら
        if word in word_freq_dic:
            # countを+1
            word_freq_dic[word] += 1
        else:
            # ない場合は、新しい単語として登録 +1カウント
            word_freq_dic.setdefault(word, 1)

    # valueで並び替える[0]にするとkeyで並び替える(降順)
    word_freq_dic = sorted(word_freq_dic.items(), key=lambda x:x[1], reverse=True)
    # データフレームを作成する
    dataframe = pd.DataFrame(word_freq_dic, columns=['word', 'count'])

    return dataframe

In [69]:
#カウントされたデータフレームがの中身を確認するときに関数
def check_dataframe_element(dataframe):
    for index, row in dataframe.iterrows():
        print(dataframe.at[index, 'word'],'count num ====', dataframe.at[index, 'count'])

In [70]:

# １人の単語を可視化するために利用する関数(ぐりぐり動かすことができるグラフ）
def create_plotly_bar_plot(dataframe):
    trace = px.bar(
        dataframe,
        x = 'count',
        y = 'word',
        width=600,
        height=800,
        title = 'word cloud use text item'

    )
    fig = trace
    fig.show()

In [71]:

#　seabornでグラフを可視化する時に利用
def create_seaborn_bar_chart(dataframe, color):
    fig = plt.subplots(figsize=(12,12))

    sns.barplot(data = dataframe,
                x = 'count',
                y = 'word',
                color = color
                )

In [72]:

# 全員まとめて実行するのかそれぞれの話者ごとで実行するのかを選択する関数
def total_or_separate(dataframe, which_play):
    dataframe_list = []
    #Tureの場合は分割して、複数人分のデータフレームを格納したリストを返す
    if which_play:
        for tag_num in dataframe['tag'].unique():
            mem_dataframe = dataframe[dataframe['tag'] == tag_num]
            dataframe_list.append(mem_dataframe)
        return dataframe_list
    # Falseの場合はそのままデータフレームを返す
    else:
        return dataframe

In [73]:
# 品詞分類〜ワードクラウドの可視化をまとめて関数化（全体用と個人用で少し動作を変更するため）
def separation_to_create_cloud(dataframe, name):

    part_word = choose_part_of_speech(dataframe, '名詞') # ワードクラウドで書き起こしたい品詞の列をリストとして取得してくる
    # 現状はここで品詞を指定
    part_word = join_list_str(part_word)

    part_word = drop_stop_words(part_word, create_stop_words()) # ストップワードを除外する

    count_df = count_word(part_word) #ここで単語のカウントを実施する(word : countの状態で記録している)
    # check_dataframe_element(count_df) # count_dfが確認したい時に実行してください

    part_word = join_list_str(part_word) # word_cloudに食べさせても問題のない形に整形する
    create_word_cloud(part_word, name)

    # create_plotly_bar_plot(count_df) # plotlyによる可視化

    count_df = count_df[count_df['count'] >= 3] # 全て可視化すると単語が潰れてしまうため出現数が3回以上の単語に限定
    create_seaborn_bar_chart(count_df, 'green') # seabornによる可視化

In [75]:
# 文字起こししたcsvファイルをもとにワードクラウドを作成して、ローカルに保存するところまでを関数化
def main():

    # 文字起こしされたcsvファイルのpath
    folder_path = '../Transcription/text_csv/*.csv'
    # ワードクラウドにつける名前を取得するためのpath
    name_path = '../Transcription/text_csv/'
    csv_path = csv_file_path_getter(folder_path)
    csv_name = csv_name_getter(name_path)
    csv_name = list(filter(None, csv_name))
    print(csv_path)
    print(csv_name)
    # 練習用
    # test_folder_path  = ['../Transcription/text_csv/transcription_ステレオaudio.csv']
    # test_csv_path = ['練習']

    for path, save_name in zip(csv_path, csv_name):

        separate = False # Trueなら別々で実行　Falseならまとめて実行


        df = create_data_frame(path) # データフレームをある程度整形した状態で作成
        word_list = word_separation_sentence_getter(df) # テキストを品詞分類して、返す（名詞、動詞、形容詞）
        print(len(word_list))
        df = append_dataframe(df, word_list) # データフレームに品詞列を結合させる
        print('dataframe length is ===', len(df))

        df = total_or_separate(df, separate) # 複数人に分割 or 全員まとめて
        if separate:
            for mem_df in df:
                separation_to_create_cloud(mem_df, save_name)
        else:
            separation_to_create_cloud(df, save_name)

        return df

In [ ]:
df_test = main()

In [77]:
# ngramのための試行錯誤コード
def test_fig(dff):
    # 積み上げグラフでみると特徴があるかも？
    test_data = []
    for mem in dff['tag'].unique():
        df_mem = dff[dff['tag'] == mem]
        part_word = choose_part_of_speech(df_mem, '名詞')
        part_word = join_list_str(part_word)
        part_word = drop_stop_words(part_word, create_stop_words())
        count_df = count_word(part_word)
        count_df['tag'] = mem
        display(count_df.head())
        test_data.append(count_df)

    df1 = test_data[0]
    df2 = test_data[1]
    df_concat = pd.concat([df1, df2])

    df1 = df1.sort_values('count')
    df2 = df1.sort_values('count')


    fig = plt.figure(figsize=(10,30))
    plt.subplots_adjust(wspace=0.4)
    ax1 = fig.add_subplot(1,2,1)
    ax1.barh(df1['word'], df1['count'],color='red')
    ax1.set_xlabel('出現回数')
    ax1.set_ylabel('名詞リスト')
    ax2 = fig.add_subplot(1,2,2)
    ax2.barh(df2['word'], df2['count'],color='blue')
    ax2.set_xlabel('出現回数')
    ax2.set_ylabel('名詞リスト')
    fig.show()

In [ ]:
# test_fig(df_test)

